# 1-行星数据
    包含天文学家观测到的围绕恒星运转的行星数据（通常简称为太阳系外行星或外行星）。
    行星数据可以直接通过 Seaborn 下载

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
# 加载行星数据
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [3]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


# 2-pandas 的简单累计功能

与一维 NumPy 数组相同，Pandas 的 Series 的累计函数也会返回一个统计值。

In [4]:
# Series 的累计函数返回一个值
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [5]:
ser.sum()

2.811925491708157

In [6]:
ser.mean()

0.5623850983416314

DataFrame 的累计函数默认对每列进行统计

In [7]:
df = pd.DataFrame({'A':rng.rand(5),
                   'B':rng.rand(5)})
df

,A,B
0,0.155995,0.020584
1,0.058084,0.969910
2,0.866176,0.832443
3,0.601115,0.212339
4,0.708073,0.181825


In [8]:
df.mean()

A    0.477888
B    0.443420
dtype: float64

In [9]:
# 设置 axis 参数，你就可以对每一行进行统计了
df.mean(axis=1)

0    0.088290
1    0.513997
2    0.849309
3    0.406727
4    0.444949
dtype: float64

注意：Pandas 的 Series 和 DataFrame 支持所有 2.4 节中介绍的常用累计函数。

describe() 方法可以计算每一列的若干常用统计值

In [10]:
# 首先丢弃缺失值
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


 # Pandas的累计方法
     指标                描述
     count()             计数项
     first()、last()     第一项与最后一项
     mean()、median()    均值与中位数 
     min()、max()        最小值与最大值
     std()、var()        标准差与方差 
     mad()               均值绝对偏差（mean absolute deviation）
     prod()              所有项乘积
     sum()               所有项求和

# 3-GroupBy: 分割、应用和组合
    对某些标签或索引的局部进行累计分析，这时就需要用到 groupby 了。

01-分割、应用和组合

In [11]:
df = pd.DataFrame({'key':['A', 'B', 'C', 'A', 'B', 'C'],
                  'data':range(6)}, columns=['key','data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


用 DataFrame 的 groupby() 方法进行绝大多数常见的 分割 - 应用 - 组合 操作

In [12]:
# 将需要分组的列名传进去即可
df.groupby('key')

注意：这里的返回值不是一个 DataFrame 对象，而是一个 DataFrameGroupBy 对象。

可以将它看成是一种特殊形式的 DataFrame，里面隐藏着若干组数据，但是在没有应用累计函数之前不会计算。

In [13]:
# 对 DataFrameGroupBy 对象应用累计函数，
# 它会完成相应的应用 / 组合步骤并生成结果。
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


02-GroupBy 对象
    
    GroupBy 对象是一种非常灵活的抽象类型。
    在大多数场景中，可以将它看成是 DataFrame 的集合，在底层解决所有难题。

GroupBy 中最重要的操作可能就是 aggregate（累计）、filter（过滤）、transform（转换） 和 apply（应用）

001--按列取值。
    
    GroupBy 对象与 DataFrame 一样，也支持按列取值，并返回一个修改过的 GroupBy 对象。

In [14]:
planets.groupby('method')

In [15]:
planets.groupby('method')['orbital_period']

这里从原来的 DataFrame 中取某个列名作为一个 Series 组。

与 GroupBy 对象一样，直到运行累计函数，才会开始计算。

In [16]:
# 获得不同方法下所有行星公转周期（按天计算）的中位数。
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

002--按组迭代

    GroupBy 对象支持直接按组进行迭代，返回的每一组都是 Series 或 DataFrame。

In [17]:
for (method, group) in planets.groupby('method'):
    print('{0:30s} shape={1}'.format(method, group.shape))

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


003-调用方法
    
    借助 Python 类可以让任何不由 GroupBy 对象直接实现的方法直接应用到每一组，无论是 DataFrame 还是 Series 对象都同样适用。

In [18]:
# 用 DataFrame 的 describe() 方法进行累计，对每一组数据进行描述性统计
planets.groupby('method')['year'].describe()

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Astrometry,2.0,2011.500000,2.121320,2010.0,2010.75,2011.5,2012.25,2013.0
Eclipse Timing Variations,9.0,2010.000000,1.414214,2008.0,2009.00,2010.0,2011.00,2012.0
Imaging,38.0,2009.131579,2.781901,2004.0,2008.00,2009.0,2011.00,2013.0
Microlensing,23.0,2009.782609,2.859697,2004.0,2008.00,2010.0,2012.00,2013.0
Orbital Brightness Modulation,3.0,2011.666667,1.154701,2011.0,2011.00,2011.0,2012.00,2013.0
Pulsar Timing,5.0,1998.400000,8.384510,1992.0,1992.00,1994.0,2003.00,2011.0
Pulsation Timing Variations,1.0,2007.000000,NaN,2007.0,2007.00,2007.0,2007.00,2007.0
Radial Velocity,553.0,2007.518987,4.249052,1989.0,2005.00,2009.0,2011.00,2014.0
Transit,397.0,2011.236776,2.077867,2002.0,2010.00,2012.0,2013.00,2014.0


03--累计、过滤、转换和应用

In [19]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key':['A','B','C','A','B','C'],
                  'data1':range(6),
                  'data2':rng.randint(0, 10, 6)},
                 columns = ['key','data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


001--累计
    
    比较熟悉的 GroupBy 累计方法只有 sum() 和 median() 之类的简单函数，
    但是 aggregate() 其实可以支持更复杂的操作，比如字符串、函数或者函数列表，并且能一次性计算所有累计值。

In [20]:
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

In [21]:
# 通过 Python 字典指定不同列需要累计的函数
df.groupby('key').aggregate({'data1': 'min',
                            'data2': 'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


In [22]:
df.groupby('key').aggregate({'data1':['min', 'max']})

data1    
      min max
key          
A       0   3
B       1   4
C       2   5

002--过滤

    过滤操作可以让你按照分组的属性丢弃若干数据

In [23]:
# 保留标准差超过某个阈值的组
def filter_func(x):
    return x['data2'].std() > 4

print(df);
print(df.groupby('key').std());
print(df.groupby('key').filter(filter_func))

  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9
       data1     data2
key                   
A    2.12132  1.414214
B    2.12132  4.949747
C    2.12132  4.242641
  key  data1  data2
1   B      1      0
2   C      2      3
4   B      4      7
5   C      5      9


filter() 函数会返回一个布尔值，表示每个组是否通过过滤。由于 A 组 'data2' 列的标准差不大于 4，所以被丢弃了。

003--转换

    累计操作返回的是对组内全量数据缩减过的结果，
    而转换操作会返回一个新的全量数据。
    数据经过转换之后，其形状与原来的输入数据是一样的。

In [24]:
# 将每一组的样本数据减去 各组的均值，实现数据标准化
df.groupby('key').transform(lambda x:x-x.mean())

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


004--apply() 方法

    apply() 方法让你可以在每个组上应用任意方法。
    输入一个 DataFrame，返回一个 Pandas 对象（DataFrame 或 Series）或一个标量（scalar，单个数值）.

In [25]:
# 用 apply() 方法将第一列数据以第二列的和为基 数进行标准化
def norm_by_data2(x):
    # x 是一个分组数据的DataFrame
    x['data1'] /= x['data2'].sum()
    return x

print(df);
print(df.groupby('key').apply(norm_by_data2))

  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9
  key     data1  data2
0   A  0.000000      5
1   B  0.142857      0
2   C  0.166667      3
3   A  0.375000      3
4   B  0.571429      7
5   C  0.416667      9


GroupBy 里的 apply() 方法非常灵活，唯一需要注意的地方是它总是输入分组数据的 DataFrame，返回 Pandas 对象或标量.

04--设置分割的键

    前面的例子一直用列名分割 DataFrame，只是其中一种方法。

001--将列表、数组、Series 或索引作为分组键。

    分组键可以是长度与 DataFrame 匹配的任意 Series 或列表。

In [26]:
# L 是新的索引，按着这个列表重新分组
L = [0,1,0,1,2,0]
print(df)
print(df.groupby(L).sum())

  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9
   data1  data2
0      7     17
1      4      3
2      4      7


In [27]:
# 还有一种比前面直接用列名更啰嗦的表示方法 df.groupby('key')
print(df.groupby(df['key']).sum())

     data1  data2
key              
A        3      8
B        5      7
C        7     12


002--用字典或 Series 将索引映射到分组名称。

    提供一个字典，将索引映射到分组键。

In [28]:
df2 = df.set_index('key')
mapping = {'A':'vowel', 'B':'consonant','C':'consonant'}
print(df2);
print(df2.groupby(mapping).sum())

     data1  data2
key              
A        0      5
B        1      0
C        2      3
A        3      3
B        4      7
C        5      9
           data1  data2
consonant     12     19
vowel          3      8


003--任意 Python 函数

    可以将任意 Python 函数传入 groupby，函数映射到索引，然后新的分组输出。

In [29]:
print(df2.groupby(str.lower).mean())

   data1  data2
a    1.5    4.0
b    2.5    3.5
c    3.5    6.0


004--多个有效键构成的列表

    任意之前有效的键都可以组合起来进行分组，从而返回一个多级索引的分组结果

In [30]:
df2.groupby([str.lower, mapping]).mean()

,,data1,data2
a,vowel,1.5,4.0
b,consonant,2.5,3.5
c,consonant,3.5,6.0


05--分组案例

In [31]:
decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0
